# Get data from Tropomi (S5P) satellite and data analisis

In this Notebook we will develop a code for get an anaice data from S5p also we will 
learn abot print thos data at gmap usin lbraries like Geemap, geopandas and Earth engine api.

sugegsted links: 
- https://courses.spatialthoughts.com/end-to-end-gee.html#earth-engine-objects


First we eil import and initializate the GEE api

In [2]:
import ee
import geemap
import geopandas 
from ipyleaflet import GeoJSON
from zipfile import ZipFile
ee.Initialize()

Now we will create a ImageCollection from S5P sensor, you could choose beetwen the following 
layers:


In [53]:
# Aviable layers:

methane = "COPERNICUS/S5P/OFFL/L3_CH4"

image = (ee.ImageCollection(methane).
         select("CH4_column_volume_mixing_ratio_dry_air_bias_corrected").
         filterDate('2022-01-01', '2023-01-01'))

Map = geemap.Map()

In [80]:
extrema = image.reduce(ee.Reducer.mean())
plane_data=extrema.reduceToVectors()
plane_data.getInfo()
data=ee.batch.Export.table.toDrive(plane_data,fileFormat='GeoJSON')
# data?

EEException: Image.reduceToVectors: Provide 'geometry' parameter when aggregating over an unbounded image.

In [81]:
datai=image.toArray()
plane_data=datai.reduceToVectors()
print(plane_data.getArray('name'))

ee.Array({
  "functionInvocationValue": {
    "functionName": "Array",
    "arguments": {
      "values": {
        "functionInvocationValue": {
          "functionName": "Element.getArray",
          "arguments": {
            "object": {
              "functionInvocationValue": {
                "functionName": "Image.reduceToVectors",
                "arguments": {
                  "image": {
                    "functionInvocationValue": {
                      "functionName": "ImageCollection.toArray",
                      "arguments": {
                        "collection": {
                          "functionInvocationValue": {
                            "functionName": "Collection.filter",
                            "arguments": {
                              "collection": {
                                "functionInvocationValue": {
                                  "functionName": "Collection.map",
                                  "arguments": {
                      

import a shapefile and create his layer:
https://geemap.org/notebooks/65_vector_styling/#use-add_styled_vector

In [19]:
departamentos = ee.FeatureCollection('users/hmojica/data/departamentos')
style = {'color': '000000', 'width': 2, 'lineType': 'solid', 'fillColor': '00000000'}
Map.addLayer(departamentos.style(**style), {}, "Departamentos")



#### ee.ImageCollection.mean
Reduces an image collection by calculating the mean of all values at each pixel across the stack of all matching bands. Bands are matched by name.

In [9]:
band_viz = {
    'min': 1750,
    'max': 1900,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}
Map.addLayer(image.mean(), band_viz, 'methane')
Map.setCenter(-74.297333, 4.570868, 5);

# Se define la region de interes
xMin = -65.654297
yMin = -4.954507
xMax = -82.573242
yMax = 13.330474
region = ee.Geometry.Rectangle([
    [xMin, yMin],
    [xMax, yMax]
  ])

In [90]:
# se obtiene las coordenadas de la región dibujndo la region con un poligono
#region = Map.user_roi #.getInfo()
#print(region)
mean=image.mean()
type(mean.toArray())

ee.image.Image

In [11]:
#Map.addLayer(image.mean(), band_viz, 'S5P Cloud')
#geemap.clip_image(Map, region, Map)


El resultado de dibujar el poligono y obtener las coordenadas es un json:

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -82.573242,
              -4.954507
            ],
            [
              -82.573242,
              13.330474
            ],
            [
              -65.654297,
              13.330474
            ],
            [
              -65.654297,
              -4.954507
            ],
            [
              -82.573242,
              -4.954507
            ]
          ]
        ]
      }
    }
  }
})


In [14]:
# Mosaic the visualization layers and display (or export).
mosaic = image.mosaic()
# Map.addLayer(mosaic, {}, 'mosaic', False)

# Create a circle by drawing a 20000 meter buffer around a point.
clipped = mosaic.clip(region)

# Display a clipped version of the mosaic.
band_viz = {
    'min': 0,
    'max': 0.95,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}
Map.addLayer(clipped, band_viz, 'Clipped image')

# Display the map
Map

Map(bottom=4292.0, center=[4.570868, -74.297333], controls=(WidgetControl(options=['position', 'transparent_bg…

In [12]:
ee.Export?

Object `ee.Export` not found.
